<span class='note'>*Make me look good.* Click on the cell below and press <kbd>Ctrl</kbd>-<kbd>Enter</kbd>.</span>

In [1]:
from IPython.core.display import HTML
HTML(open('css/custom.css', 'r').read())

<h5 class='prehead'>SA367 &middot; Mathematical Models for Decision Making &middot; Spring 2021 &middot; Uhan</h5>

<h5 class='lesson'>Lesson 8.</h5>

<h1 class='lesson_title'>Drafting a fantasy basketball team</h1>

## The problem

You're preparing for your upcoming fantasy basketball draft. You wonder: what is the best possible team you can draft?

You have the following data:

* Projected __auction prices__ for each player in the NBA.
* The __z-score__ for each player: the sum of the number of standard deviations above the mean in the following 9 categories:
    1. points per 36 minutes
    2. 3 point field goals made per 36 minutes
    3. number of rebounds per 36 minutes
    4. number of assists per 36 minutes
    5. number of steals per 36 minutes
    6. number of blocks per 36 minutes
    7. _negative_ of the number of turnovers per 36 minutes
    8. field goal percentage
    9. free throw percentage
    
Your roster must have exactly 12 players, and you have a budget of \$50. You want to maximize the total z-score of your team.

Formulate this problem as a dynamic program by giving its shortest/longest path representation.

## Setting up the data

* In the same folder as this notebook, there is a file called `fantasy_basketball_nba2017.csv` with the data described above.
    - The z-scores were computed using projected stats from [Basketball Reference](http://www.basketball-reference.com/friv/projections.cgi).
    - Projected auction prices were taken from [Yahoo! Fantasy Sports](https://basketball.fantasysports.yahoo.com/nba/draftanalysis?tab=AD&pos=ALL&sort=DA_AP), normalized to a budget of \$50.
    

*  Let's take a look using pandas. First, let's import pandas:

In [2]:
# Import pandas
import pandas as pd

* Now we can read the `csv` file into a pandas DataFrame and inspect the first few rows:

In [3]:
# Read csv file with data
df = pd.read_csv('fantasy_basketball_nba2017.csv')

# Print the first 5 rows of df
df.head()

,PLAYER,TEAM,POSITIONS,ZSCORE,PRICE
0,Stephen Curry,GS,"PG,SG",12.681705,18
1,Kawhi Leonard,SA,"SG,SF",8.994709,16
2,Chris Paul,LAC,PG,8.485619,15
3,Anthony Davis,NO,"PF,C",8.357714,15
4,Kevin Durant,GS,"SF,PF",7.848493,18


* As we can see, the data also contains the team and positions for each player.

* Next, let's create some lists that correspond to the relevant columns of the dataset.

* Recall that we can grab a column from a DataFrame like this:

```python
df['COLUMN_NAME']
```

* The `list()` function turns any list-like object (such as a column of a pandas DataFrame) into a Python list.

* We can apply the `.str.split(",")` method to convert a comma-delimited string into a list. This will be helpful in parsing the positions that a player can play, since many players can play multiple positions.

In [4]:
# Create a list of players
players = list(df["PLAYER"])

# Create a list of zscores
zscores = list(df["ZSCORE"])

# Create a list of prices
prices = list(df["PRICE"])

# Create a list of positions
positions = list(df["POSITIONS"].str.split(","))

* Now we can look at player $t$ and his associated data like this: 

In [5]:
# Print out information about player 3 - Anthony Davis
print(players[3])
print(zscores[3])
print(prices[3])
print(positions[3])

Anthony Davis
8.357714143615143
15
['PF', 'C']


* Let's also create a variable that holds the number of players:

In [6]:
# Create a variable for the number of players
n_players = len(players)

* Now we can use these lists and variables to construct the graph for the dynamic program.

## Solving the DP

* There are two important constants in our problem: the budget, and the roster size. 

* Let's create variables to hold these constants.

* This way, we can easily adapt our code to accomodate similar DPs with different budgets and roster sizes.

In [7]:
# Create variables to hold constants: budget, roster size
BUDGET = 50
ROSTER_SIZE = 12

* Next, let's import `networkx` and `bellmanford`:

In [8]:
# Import networkx and bellman ford
import networkx as nx
import bellmanford as bf

* As usual, we start with an empty graph:

In [9]:
# Create empty digraph
G = nx.DiGraph()

* Next, let's add the nodes:

In [10]:
# Add stage-state nodes (t, n1, n2)
for t in range(0, n_players + 1):
    for n1 in range(0, BUDGET + 1):
        for n2 in range(0, ROSTER_SIZE + 1):
            G.add_node((t, n1, n2))

# Add the end node
G.add_node("end")

* How many nodes do we have in our graph?

In [11]:
# Print number of nodes in digraph
print(G.number_of_nodes())

293710


* Now it's time to add the edges.

* Let's start with the edges corresponding to the decision of whether to take a player or not:

In [12]:
# Add edges corresponding to the decision of whether to take a player or not
for t in range(0, n_players):
    for n1 in range(0, BUDGET + 1):
        for n2 in range(0, ROSTER_SIZE + 1):
            
            # Don't take the player
            G.add_edge((t, n1, n2), (t + 1, n1, n2), length=0)

            # Take the player if there's enough left in the budget
            # and there are enough roster spots
            if n1 - prices[t] >= 0:
                if n2 - 1 >= 0:
                    G.add_edge((t, n1, n2), (t + 1, n1 - prices[t], n2 - 1), length=-zscores[t])

* Now we can add the edges from the last stage to the end node. Remember to only add edges from the last stage if the number of remaining roster spots $n_2$ is equal to 0!

In [13]:
# Add edges from last stage to end, 
# only when number of remaining roster spots is 0
for n1 in range(0, BUDGET + 1):
    G.add_edge((n_players, n1, 0), "end", length=0)

* How many edges do we have in our graph?

In [14]:
# Print number of edges
print(G.number_of_edges())

550545


* Finally, let's solve the shortest path problem we've constructed using the Bellman-Ford algorithm:

In [15]:
# Solve the shortest path problem using the Bellman-Ford algorithm
length, nodes, negative_cycle = bf.bellman_ford(G, source=(0, BUDGET, ROSTER_SIZE), target="end", 
                                                weight="length")

print(f"Negative cycle? {negative_cycle}")
print(f"Shortest path length: {length}")
print(f"Shortest path: {nodes}")

Negative cycle? False
Shortest path length: -57.45369330886113
Shortest path: [(0, 50, 12), (1, 32, 11), (2, 32, 11), (3, 32, 11), (4, 32, 11), (5, 32, 11), (6, 32, 11), (7, 32, 11), (8, 32, 11), (9, 23, 10), (10, 15, 9), (11, 15, 9), (12, 14, 8), (13, 14, 8), (14, 14, 8), (15, 14, 8), (16, 14, 8), (17, 14, 8), (18, 13, 7), (19, 13, 7), (20, 13, 7), (21, 13, 7), (22, 13, 7), (23, 13, 7), (24, 12, 6), (25, 9, 5), (26, 9, 5), (27, 9, 5), (28, 5, 4), (29, 5, 4), (30, 5, 4), (31, 5, 4), (32, 5, 4), (33, 5, 4), (34, 5, 4), (35, 5, 4), (36, 5, 4), (37, 5, 4), (38, 5, 4), (39, 5, 4), (40, 5, 4), (41, 3, 3), (42, 3, 3), (43, 3, 3), (44, 3, 3), (45, 3, 3), (46, 3, 3), (47, 3, 3), (48, 2, 2), (49, 2, 2), (50, 2, 2), (51, 2, 2), (52, 1, 1), (53, 0, 0), (54, 0, 0), (55, 0, 0), (56, 0, 0), (57, 0, 0), (58, 0, 0), (59, 0, 0), (60, 0, 0), (61, 0, 0), (62, 0, 0), (63, 0, 0), (64, 0, 0), (65, 0, 0), (66, 0, 0), (67, 0, 0), (68, 0, 0), (69, 0, 0), (70, 0, 0), (71, 0, 0), (72, 0, 0), (73, 0, 0), (74, 0, 

* It's easy to see what the maximum possible total z-score is... however, which players should we select to get this maximum total z-score?

* Instead of reading through the path of 400+ nodes to figure out which players to select, let's write some code to do this for us.

* We know that we select a player whenever the number of remaining roster spots $n_2$ goes down by 1 from stage to stage. So...

In [16]:
# Print selected players in a more user-friendly format
# Get number of nodes in shortest path
n_nodes = len(nodes)

# Go through each node in the shortest path
for i in range(n_nodes - 2):
    
    # Node in current stage
    (t, n1, n2) = nodes[i]
    
    # Node in next stage
    (next_t, next_n1, next_n2) = nodes[i + 1]
    
    # If n2 isn't the same from one stage to the next, print the player's info
    if n2 != next_n2:
        print(f"Node: {nodes[t]}  Player: {players[t]}  Positions: {positions[t]}, Price: {prices[t]}  Z-Score: {zscores[t]}")

Node: (0, 50, 12)  Player: Stephen Curry  Positions: ['PG', 'SG'], Price: 18  Z-Score: 12.681704920021767
Node: (8, 32, 11)  Player: Nikola Jokic  Positions: ['PF', 'C'], Price: 9  Z-Score: 6.245534045281088
Node: (9, 23, 10)  Player: Klay Thompson  Positions: ['SG', 'SF'], Price: 8  Z-Score: 5.781494181785728
Node: (11, 15, 9)  Player: Cole Aldrich  Positions: ['C'], Price: 1  Z-Score: 5.689641521236912
Node: (17, 14, 8)  Player: Boban Marjanovic  Positions: ['C'], Price: 1  Z-Score: 4.542112513644865
Node: (23, 13, 7)  Player: Brandan Wright  Positions: ['PF', 'C'], Price: 1  Z-Score: 4.092581777062199
Node: (24, 12, 6)  Player: Jrue Holiday  Positions: ['PG'], Price: 3  Z-Score: 4.0156102748746365
Node: (27, 9, 5)  Player: Dirk Nowitzki  Positions: ['PF', 'C'], Price: 4  Z-Score: 3.798143244965448
Node: (40, 5, 4)  Player: Kelly Olynyk  Positions: ['C'], Price: 2  Z-Score: 2.9855612991757123
Node: (47, 3, 3)  Player: Jeremy Lamb  Positions: ['SG', 'SF'], Price: 1  Z-Score: 2.6579023

## Incorporating other roster constraints

* Fantasy basketball leagues usually have some roster constraints &mdash; in particular, on player positions.

* For example, suppose our roster must have exactly 2 players that can play center (C).

* How can we modify our dynamic program to accomodate this? Write a new dynamic program on paper.

* How do we need to modify the code above to solve the new dynamic program?

* A hint:

    - To check if player $t$ can play center, we can write:

    ```python
    if "C" in positions[t]:
        ...
    ```

    - This code does what it looks like: it checks if `"C"` is in the list of positions `positions[t]` that player $t$ can play.

In [17]:
# Create empty digraph
H = nx.DiGraph()

# Add stage-state nodes (t, n1, n2, n3)
# t = player
# n1 = remaining budget
# n2 = remaining roster spots
# n3 = remaining C roster spots
for t in range(0, n_players):
    for n1 in range(0, BUDGET + 1):
        for n2 in range(0, ROSTER_SIZE + 1):
            for n3 in range(0, 3):
                G.add_node((t, n1, n2, n3))

# Add the end node
H.add_node("end")

# Add edges corresponding to the decision of whether to take a player or not
for t in range(0, n_players):
    for n1 in range(0, BUDGET + 1):
        for n2 in range(0, ROSTER_SIZE + 1):
            for n3 in range(0, 3):
            
                # Don't take the player
                H.add_edge((t, n1, n2, n3), (t + 1, n1, n2, n3), length=0)

                # Take the player if there's enough left in the budget
                # and there are enough roster spots
                if n1 - prices[t] >= 0:
                    if n2 - 1 >= 0:
                    
                        # If the player is a center, we can only add this edge if
                        # there are enough remaining C roster spots
                        if "C" in positions[t]:
                            if n3 - 1 >= 0:
                                H.add_edge((t, n1, n2, n3), 
                                           (t + 1, n1 - prices[t], n2 - 1, n3 - 1), 
                                           length=-zscores[t])

                        # Otherwise, the number of remaining C roster spots stays the same
                        else:
                            H.add_edge((t, n1, n2, n3), (t + 1, n1 - prices[t], n2 - 1, n3), 
                                       length=-zscores[t])

# Add edges from last stage to end, 
# only when number of remaining roster spots is 0 and
# the number of remaining C roster spots is 0
for n1 in range(0, BUDGET + 1):
    H.add_edge((n_players, n1, 0, 0), "end", length=0)


# Solve the shortest path problem using the Bellman-Ford algorithm    
length, nodes, negative_cycle = bf.bellman_ford(H, source=(0, BUDGET, ROSTER_SIZE, 2), 
                                                target="end", weight="length")

print(f"Negative cycle? {negative_cycle}")
print(f"Shortest path length: {length}")
print(f"Shortest path: {nodes}")

# Print selected players in a more user-friendly format
# Get number of nodes in shortest path
n_nodes = len(nodes)

# Go through each node in the shortest path
for i in range(n_nodes - 2):
    
    # Node in current stage
    (t, n1, n2, n3) = nodes[i]
    
    # Node in next stage
    (next_t, next_n1, next_n2, next_n3) = nodes[i + 1]
    
    # If n2 isn't the same from one stage to the next, print the player's info
    if n2 != next_n2:
        print(f"Node: {nodes[t]}  Player: {players[t]}  Positions: {positions[t]}, Price: {prices[t]}  Z-Score: {zscores[t]}")

Negative cycle? False
Shortest path length: -54.39369050175551
Shortest path: [(0, 50, 12, 2), (1, 32, 11, 2), (2, 32, 11, 2), (3, 17, 10, 2), (4, 17, 10, 2), (5, 17, 10, 2), (6, 17, 10, 2), (7, 17, 10, 2), (8, 17, 10, 2), (9, 17, 10, 2), (10, 17, 10, 2), (11, 17, 10, 2), (12, 16, 9, 1), (13, 16, 9, 1), (14, 16, 9, 1), (15, 16, 9, 1), (16, 16, 9, 1), (17, 16, 9, 1), (18, 15, 8, 0), (19, 15, 8, 0), (20, 15, 8, 0), (21, 15, 8, 0), (22, 15, 8, 0), (23, 15, 8, 0), (24, 15, 8, 0), (25, 12, 7, 0), (26, 12, 7, 0), (27, 12, 7, 0), (28, 12, 7, 0), (29, 12, 7, 0), (30, 12, 7, 0), (31, 12, 7, 0), (32, 9, 6, 0), (33, 5, 5, 0), (34, 5, 5, 0), (35, 5, 5, 0), (36, 5, 5, 0), (37, 5, 5, 0), (38, 5, 5, 0), (39, 5, 5, 0), (40, 5, 5, 0), (41, 5, 5, 0), (42, 5, 5, 0), (43, 5, 5, 0), (44, 5, 5, 0), (45, 5, 5, 0), (46, 5, 5, 0), (47, 5, 5, 0), (48, 4, 4, 0), (49, 4, 4, 0), (50, 4, 4, 0), (51, 4, 4, 0), (52, 3, 3, 0), (53, 3, 3, 0), (54, 3, 3, 0), (55, 3, 3, 0), (56, 3, 3, 0), (57, 3, 3, 0), (58, 3, 3, 0), (5

## Food for thought

* Can the dynamic programs we solved above help with an actual fantasy basketball draft? Why or why not?

<!-- _Write your notes here. Double-click to edit._ -->
* These DPs only give you the best possible roster. They don't model the draft process; in particular, not all players may be available when it's our turn to select, and the DPs don't use actual auction prices.

* These DPs can help plan _during_ a draft: as a draft progresses, one can update the DP to remove the players that have been already selected, and use the DP to plan which of the remaining players to focus on.

---

## Problems

### Problem 1 (Airlift planning)

See the accompanying PDF for this lesson.

### Problem 2 (Solving the airlift planning problem)

Solve the dynamic program your formulated for Problem 1 using NetworkX. What is the maximum possible total capability value? Which requirements should be put on the plane to achieve this maximum total capability value?

In [1]:
# Solution
# Import networkx and bellmanford
import networkx as nx
import bellmanford as bf

# Create variables for weight and volume capacity
MAX_WEIGHT = 80
MAX_VOLUME = 700 

# Create lists for capability values, weights, volumes
value = [50, 30, 80, 40, 70]
weight = [43, 17, 26, 4, 35]
volume = [250, 130, 370, 180, 400]

# Create empty graph
G = nx.DiGraph()

# Add stage-state nodes (t, n1, n2)
#   t = stage, consider requirement t
#   n1 = remaining weight
#   n2 = remaining volume
# 
# t = 0: large munitions
# t = 1: small munitions
# t = 2: food
# t = 3: medical supplies
# t = 4: repair parts
# t = 5: end of decision process
for t in range(0, 6):
    for n1 in range(0, MAX_WEIGHT + 1):
        for n2 in range(0, MAX_VOLUME + 1):
            G.add_node((t, n1, n2))

# Add end node
G.add_node("end")

# Add edges corresponding to decisions
# Remember we're maximizing, so the lengths should be negative of the values
for t in range(0, 5):
    for n1 in range(0, MAX_WEIGHT + 1):
        for n2 in range(0, MAX_VOLUME + 1):
            # Take requirement t, check if it fits first
            if n1 - weight[t] >= 0:
                if n2 - volume[t] >= 0:
                    G.add_edge((t, n1, n2), (t + 1, n1 - weight[t], n2 - volume[t]), 
                               length=-value[t])

            # Don't take requirement t
            G.add_edge((t, n1, n2), (t + 1, n1, n2), length=0)

# Add edges from last stage to end node
for n1 in range(0, MAX_WEIGHT + 1):
    for n2 in range(0, MAX_VOLUME + 1):
        G.add_edge((5, n1, n2), "end", length = 0)

# Solve the DP/shortest path problem using the Bellman-Ford algorithm
length, nodes, negative_cycle = bf.bellman_ford(G, source=(0, MAX_WEIGHT, MAX_VOLUME), 
                                                target="end", weight="length")

print(f"Negative cycle? {negative_cycle}")
print(f"Shortest path length: {length}")
print(f"Shortest path: {nodes}")

Negative cycle? False
Shortest path length: -150
Shortest path: [(0, 80, 700), (1, 80, 700), (2, 63, 570), (3, 37, 200), (4, 33, 20), (5, 33, 20), 'end']


* The maximum possible total capability value is 150.

* To achieve this maximum total capability value, we should put the small munitions, food, and medical supplies on the C-17.